## ancillary_LES_data.ipynb

Calculates and writes out additional 2D ancillary fields for each hour of each LES simulation:
* stability regime
* cloud base (pressure and height AGL)
* PBL top (pressure and height AGL) calculated using theta and theta_v
* LCL pressure, height, temperature
* LFC pressure, height, temperature
* EL pressure, height, temperature
* CIN
* CAPE

The expanded variable list is written to pbl2c_YYYYMMDD_SS.nc4
* YYYY = 4-digit year
* MM   = 2-digit month
* DD   = 2-digit day
* SS   = set of lower boundary conditions
    * 00   = heterogeneous BCs from HydroBlocks (HET)
    * 01   = homogeneous (HOM; domain mean applied at all grid cells) 


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import metpy.calc as calc
from metpy.units import units
import metpy.constants as const

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import datetime as dt
import time
import sys, warnings, glob, gc
warnings.filterwarnings("ignore", message="All-NaN slice encountered")


## Ancillary fields from LES model output

In [ ]:
#ddir = "/project/land/WRF-HB/LES_runs/chaney-web-00.egr.duke.edu/CLASP/LES/diags/" # Orig data retrieved by Finley
#exp_list = sorted(glob.glob(ddir+"/doz_*_00"))   # Orig data retrieved by Finley

suffix = ".nc4"
in_prefix = "fr2"
prefix = "pbl2c"
offset = 33

bcs = "_00"    # Heterogeneous
bcs = "_01"    # Homogeneous
ddir = "/Volumes/SSD_8TB/CLASP/LES_runs2/"                  # Directory of original LES output
pdir = "/Volumes/SSD_8TB/CLASP/LES_HCF/PBL_data_new/"       # Directory for the anicllary field files produced here
exp_list = sorted(glob.glob(f"{ddir}/{in_prefix}*{bcs}*"))  # Sorted list of the available cases for the chosen set of BCs

hours = ["13","14","15","16","17","18","19","20","21","22","23","00","01","02","03"] # Which hours to process and save
dom = [31,28,31,30,31,30,31,31,30,31,30,31]     # Days in each month

ex0,ex1 = 0,92  # RAnge of cases to process (useful to restart if processing is interrupted)

# For deflated NetCDF4 output:
deflate = dict(zlib=True, complevel=1)

####################
#### Parameters ####
####################
# For estimation of boundary layer height from potential temperature only
delta_s   = 1.0     # [K]     -- from Table 1 of doe-sc-arm-tr-132.pdf
delta_u   = 0.5     # [K]     -- from Table 1 of doe-sc-arm-tr-132.pdf
delta_p   = 1500    # [Pa]    -- Difference between 5 hPa AGL and 20 hPa AGL implied for the two deltas above
overshoot = 4.0e-3  # [K/m]   -- from Table 1 of doe-sc-arm-tr-132.pdf
stab_curv = -4.0e-5 # [K/m^2] -- They seem to have units wrong here - I think this is correct.
min_height = 150    # [m]     -- Essentially 6th layer up [5:]
level_min  = 5

min_qc     = 1.0e-5 # [kg/kg] -- Trheshold for determining presence of cloud - somewhat arbitrary

####################
#### Constants  ####
####################
t_0 = 273.15               # [K]
R_d = const.Rd.m           # [J/kg/K]
c_p = const.Cp_d.m         # [J/kg/K]
L_v = const.Lv.m           # [J/kg]
epsilon = const.epsilon.m  # [-] 
p_triple = 611.2           # [Pa]

print(len(exp_list))
print(exp_list)

## Define functions to use

In [ ]:

def cloud_base(cloud_mixrat,pressure,height,cloud_frac=1.0,min_qc=1.0e-5):
    """
    Returns the height (AGL) and pressure of the cloud base given profiles
    of cloud water mixing ratio
        
    Required inputs:
        cloud_mixrat    = (array) Profile of cloud liquid water mixing ratio [kg/kg]
        pressure        = (array) Profile of barometric pressure [Pa]
        height          = (array) Heights profile (geopotential or AGL) corresponding to pressure levels [m]
    
    Optional inputs:
        cloud_frac      = (scalar or array) Fractional cloud cover (non-zero when there is subgrid cloud in the model parameterization)
        min_qc          = (scalar or array) Minimum value of liquid water mixing ratio for cloud [default=1.0e-5 kg/kg]
        
    Outputs:
        cb_pressure     = (scalar or array) Pressure at cloud base [Pa]
        cb_height       = (scalar or array) Height (AGL) of cloud base [m]
    """

    vdim = pressure.dims[0]
    shifts = {vdim: 1}
    eps = 1e-25
    
    # Cloud base estimated
    #qc_dif = (cloud_mixrat - min_qc)  # Positive for high enough liquid water concentration to be defined as cloud
    qc_dif = (np.log((cloud_mixrat/cloud_frac).where(cloud_frac>0,0) + eps) - np.log(min_qc + eps))  # Changed to better represent sharp base, not smear it down into layer below

    ddif = qc_dif.diff(vdim,n=1,label='lower')      # Intervals of differences, for interpolation
    dp = pressure.diff(vdim,n=1,label='lower')      # Pressure intervals between vertical levels

    b_cld = np.heaviside(qc_dif,1)                  # 1 in clouds, 0 where not
    s_tran = np.heaviside(qc_dif,1).diff(vdim,n=1,label='lower')  # 1 below transition to cloud, -1 opposite, 0 elsewhere
    iii = np.abs(s_tran).sum(dim=vdim)              # Total number of transitions between cloud and clear
    l_cb = np.abs(s_tran)*((-np.abs(s_tran).cumsum(dim=vdim) + iii)/(iii-1)).astype('int')
                                                    # 1 just below cloud base(s), -1 below cloud top(s), 0 elsewhere
    a_cb = l_cb.shift(shifts,fill_value=0.0)        # 1 just above cloud base(s), -1 above cloud top(s), 0 elsewhere
    l_wgt = (l_cb*(ddif+qc_dif[:-1])/ddif).fillna(0)                     # Weight to pressure level below EL
    weights = l_wgt + a_cb * (1-l_wgt).shift(shifts,fill_value=0)        # Weights at the layer above and below only
    cb_pressure = np.exp((np.log(pressure[:-1])*weights).sum(dim=vdim))  # Pressure at cloud base
    cb_pressure = cb_pressure*iii/iii                                    # Set 0 to nan
    cb_height = (height[:-1]*weights).sum(dim=vdim)                      # Height (AGL) of cloud base
    cb_height = cb_height*iii/iii                                        # Set 0 to nan
    
    del(qc_dif,ddif,dp,b_cld,s_tran,iii,l_cb,a_cb,l_wgt,weights)
    gc.collect()    
    
    return cb_pressure,cb_height


def pbl_from_pottemp(pressure,theta,height,
                     delta_s=1.0,delta_u=0.5,delta_p=1500,overshoot=4.0e-3):
    """
    Find PBL top level from profiles of potential temperature only
    Based on [Liu & Liang (2010)]
             (https://doi.org/10.1175/2010JCLI3552.1) 
    and [Sivaraman et al. (2013)]
             (https://www.arm.gov/publications/tech_reports/doe-sc-arm-tr-132.pdf)
    The steps are:
    * Interpolate potential tepmperature in the vertical to 5 & 20 hPa AGL
    * Find vertical gradient of potential temperature
    * Determine regime (stable, neutral, unstable)
    * Conditionally determine PBL height based on vertical gradients 
    
    It is assumed that the lowest level of input profiles is the "surface" from which to start parcel ascent.
    It is also assumed that the first dimension in arrays is the vertical dimension
    Can be applied to virtual potential temperature profiles as well 
    
    Required inputs:
        pressure    = (array) Profile of barometric pressure [Pa]
        theta       = (array) Profile of (virtual) potential temperature [K]
        height      = (array) Heights profile (geopotential or AGL) corresponding to pressure levels [m]
    
    Optional inputs:
        delta_s     = (scalar) from Table 1 of doe-sc-arm-tr-132.pdf [default=1.0 K]
        delta_u     = (scalar) from Table 1 of doe-sc-arm-tr-132.pdf [default=0.5 K]
        delta_p     = (scalar) Difference in pressure implied for the two deltas above [default=1500 Pa]
        overshoot   = (scalar) from Table 1 of doe-sc-arm-tr-132.pdf [default=4.0e-3 K/m]
        
    Outputs:
        cb_pressure     = (scalar or array) Pressure at cloud base [Pa]
        cb_height       = (scalar or array) Height (AGL) of cloud base [m]
        regime          = (scalar or array) Convective regime: 0 = neutral
                                                               1 = stable
                                                              -1 = unstable
    """

    vdim = pressure.dims[0]
    shift2 = {vdim: 2}

    # Find lower troposphere lapse rate to determine stability regime
    lt_delta = theta.values[5]-theta.values[1]                    # Roughly 18 hPa minus 5 hPa AGL theta
    lp_delta = pressure.values[5]-pressure.values[1]              # Roughly 18 hPa minus 5 hPa AGL pressure
    regime = 0+(lt_delta/lp_delta>delta_s/delta_p)-(lt_delta/lp_delta<-delta_s/delta_p)
                                                                  #  0 = neutral
                                                                  #  1 = stable
                                                                  # -1 = unstable

    press_c = np.exp(np.log(pressure).rolling(shift2).mean())[1:] # Pressures midway between layers (linear in ln(p))
    height_c = height.rolling(shift2).mean()[1:]                  # Heights midway between layers (linear interp)
    theta_d1 = theta.diff(vdim,n=1,label='lower')/height.diff(vdim,n=1,label='lower')         # d{theta}/dz

    # Clear arrays to fill    
    pbl_pressure = np.zeros_like(pressure[0])
    pbl_height = np.zeros_like(pressure[0])

    ##### Valid for stable regime:
    level_sbl = np.heaviside(theta_d1.diff(vdim,label='lower'),0).diff(vdim,label='lower')
    level_sbl = level_sbl.where(level_sbl==1)
    # We check for inversion in two layers above the level found
    aa = 0.5*(theta_d1<overshoot).astype(int).rolling(shift2).sum()
    level_sbl = (aa.where(aa==1.))[1:-1]*level_sbl
    # The 'where' checks based on regime apply results only to SBL grid cells
    pbl_height = pbl_height + (height_c[1:-1]*level_sbl).reduce(np.nanmin,dim=vdim)*np.where(regime==1,1,0)
    pbl_pressure = pbl_pressure + (press_c[1:-1]*level_sbl).reduce(np.nanmax,dim=vdim)*np.where(regime==1,1,0)

    ##### Valid for neutral and unstable regimes:
    unstab = (theta - theta[0] - delta_u)
    tgrad = theta_d1 - overshoot
    level_ubl = np.sign(unstab[:-1].where((unstab[:-1]>0)&(tgrad>0)))
    # The 'where' checks based on regime apply results only to CBL and NRL grid cells
    pbl_height = pbl_height + (height_c*level_ubl).reduce(np.nanmin,dim=vdim)*np.where(regime<1,1,0)
    pbl_pressure = pbl_pressure + (press_c*level_ubl).reduce(np.nanmax,dim=vdim)*np.where(regime<1,1,0)

    #del(lt_delta,lp_delta,press_c,height_c,theta_d1,level_sbl,aa,unstab,tgrad,level_ubl)
    #gc.collect()    

    return pbl_pressure, pbl_height, regime
    
def regime_from_pottemp(pressure,theta,delta_s=1.0,delta_p=1500):
    """
    Find convective regime near surface (stable, neutral, unstable)
    
    It is assumed that the lowest level of input profiles is the "surface" from which to start parcel ascent.
    It is also assumed that the first dimension in arrays is the vertical dimension
    Can be applied to virtual potential temperature profiles as well 
    
    Required inputs:
        pressure    = (array) Profile of barometric pressure [Pa]
        theta       = (array) Profile of (virtual) potential temperature [K]
    
    Optional inputs:
        delta_s     = (scalar) from Table 1 of doe-sc-arm-tr-132.pdf [default=1.0 K]
        delta_p     = (scalar) Difference in pressure implied for the two deltas above [default=1500 Pa]
        
    Outputs:
        regime          = (scalar or array) Convective regime: 0 = neutral
                                                               1 = stable
                                                              -1 = unstable
    """

    # Find lower troposphere lapse rate to determine stability regime
    lt_delta = theta.values[1]-theta.values[0]                    # Between two lowest model layers theta
    lp_delta = -(pressure.values[1]-pressure.values[0])              # Between two lowest model layers pressure
    regime = 0+(lt_delta/lp_delta>delta_s/delta_p)-(lt_delta/lp_delta<-delta_s/delta_p)
                                                                  #  0 = neutral
                                                                  #  1 = stable
                                                                  # -1 = unstable
    return regime
                
                
def malr_dtdp(pressure,temperature):    # Moist adiabatic lapse rate values of dT/dp 
    sat_e = 611.2*np.exp(17.67*(temperature-t_0)/(temperature-t_0+243.5))
    sat_w = sat_e * epsilon / (pressure - sat_e)
    malr = (R_d*temperature + L_v*sat_w) / (c_p + L_v*L_v*sat_w*epsilon/(R_d*temperature*temperature)) / pressure

    del(sat_e,sat_w)
    gc.collect()

    return malr

def poisson(pressure,ref_temperature,ref_pressure=100000):
    """
    Returns the temperature at input pressure that has the same potential temperature as 
        the air at the reference temperature and pressure
        
    Required inputs:
        pressure         = (scalar or array) Target barometric pressure [Pa]
        ref_temperature  = (scalar or array) Air temperature at reference pressure [K]
    
    Optional inputs:
        ref_pressure     = (scalar or array) Reference pressure [default: 1e5 Pa]
        
    Outputs:
        temperature      = (scalar or array) Air temperature at target pressure [K]
    """
    return (pressure/ref_pressure)**(R_d/c_p) * ref_temperature

def pressure_from_heights(target_height,pressure,height):
    """
    Returns the pressure(s) corresponding to the given height(s) by interpolation [z ~ ln(p)]
    
    It is assumed that the lowest level of input profiles is the "surface" from which to start parcel ascent.
    It is also assumed that the first dimension in arrays is the vertical dimension
    
    Required inputs:
        target_height   = (scalar or array) Height to be converted to pressure [m]
        pressure        = (array) Barometric pressure profile [Pa]
        height          = (array) Heights profile (geopotential or AGL) corresponding to pressure levels [m]
        
    Outputs:
        target_pressure = (scalar or array) Pressure at the target height [Pa]
    """
    
    vdim = pressure.dims[0]
    shifts = {vdim: 1}
        
    # Find the LCL
    dif = height - target_height                           # Zero at target height, negative below, positive above
    ddif = dif.diff(vdim,n=1,label='lower')                # Intervals of differences between profiles
    dp = pressure.diff(vdim,n=1,label='lower')             # Pressure intervals between vertical levels
    ddd = np.heaviside(dif,1).diff(vdim,n=1,label='lower') # 1 at level just below target, 0 elsewhere
    eee = ddd.shift(shifts,fill_value=0.0)                 # 1 at level just above target, 0 elsewhere

    w_below = (ddd*(ddif+dif[:-1])/ddif).fillna(0)         # Weight to pressure level below target
    weights = w_below + eee * (1-w_below).shift(shifts,fill_value=0)       # Weights at the layer above and below only
    target_pressure = np.exp((np.log(pressure[:-1])*weights).sum(dim=vdim))    # Pressure at target
        
    del(dif,ddif,dp,ddd,eee,w_below,weights)
    gc.collect()

    return target_pressure

def parcel_profile(pressure,temperature,sfc_mixrat,height=None):
    """
    Returns the profile corresponding to:
    1. A dry adiabatic lapse rate between surface and lifted condensation level (LCL)
    2. A moist adiabatic lapse rate above the LCL
    Also returns information about the LCL
    
    It is assumed that the lowest level of input profiles is the "surface" from which to start parcel ascent.
    It is also assumed that the first dimension in arrays is the vertical dimension
    
    Required inputs:
        pressure        = (array) Barometric pressure profile [Pa]
        temperature     = (array) Air temperature profile [K]
        sfc_mixrat      = (scalar or array) Water vapor mixing ratio at the surface [kg/kg]
    
    Optional inputs:
        height          = (array) Heights profile (geopotential or AGL) corresponding to pressure levels [m]
        
    Outputs:
        profile         = (array) Vertical parcel temperature profile as described above [K]
        lcl_pressure    = (scalar or array) Pressure at LCL [Pa]
        lcl_temperature = (scalar or array) Temperature at LCL [K]
        
    Optional outputs:
        lcl_height      = (scalar or array) If heights are supplied as an input, height at LCL [m]
    """
    #global profile, dalr, plcl_sign, malr, vp, ccmr, dif, ddif, dp, ddd, eee, w_below, weights, lcl_pressure, lcl_temperature, mm
    
    vdim = pressure.dims[0]
    shifts = {vdim: 1}
        
    # Set up shape of profile array
    profile = xr.full_like(pressure,np.nan)

    # Find the LCL
    dalr = poisson(pressure,temperature[0],pressure[0])    # Dry adiabatic lapse rate up from lowest level temperature
    vp = pressure * sfc_mixrat / (sfc_mixrat + epsilon)    # Vapor pressure profile matching lowest level mixing ratio [Pa]
    ccmr = (np.log(vp/p_triple)*243.5)/(17.67-np.log(vp/p_triple)) + t_0   # Matching dewpoint profile [K] (from Bolton 1980)
    
    dif = (ccmr - dalr)                                    # Zero at LCL
    ddif = dif.diff(vdim,n=1,label='lower')                # Intervals of differences between profiles
    dp = pressure.diff(vdim,n=1,label='lower')             # Pressure intervals between vertical levels
    ddd = np.heaviside(dif,1).diff(vdim,n=1,label='lower') # 1 at level just below LCL, 0 elsewhere
    eee = ddd.shift(shifts,fill_value=0.0)                 # 1 at level just above LCL, 0 elsewhere
    w_below = (ddd*(ddif+dif[:-1])/ddif).fillna(0)         # Weight to pressure level below LCL
    weights = w_below + eee * (1-w_below).shift(shifts,fill_value=0)       # Weights at the layer above and below only
    lcl_pressure = np.exp((np.log(pressure[:-1])*weights).sum(dim=vdim))   # Pressure at LCL
    lcl_temperature = poisson(lcl_pressure,temperature[0],pressure[0])     # Temperature at LCL
    
    # Parcel profile below LCL is determined - find profile above
    #plcl_sign = (1 - np.sign(pressure-lcl_pressure,0))/2                  # 1 above LCL, 0 below
    plcl_sign = (1 - np.heaviside(pressure-lcl_pressure,0))                # 1 above LCL, 0 below
    mm = (plcl_sign*malr_dtdp(pressure,temperature))[:-1] * dp             # Moist adiabatic temperature intervals 
    malr = (mm.cumsum(dim=vdim) + lcl_temperature) * plcl_sign[:-1]        # Moist adiabat above LCL at each X,Y
    profile[:-1] = dalr[:-1]*(1-plcl_sign[:-1]) + malr

    if height.all() == None:
        lcl_height = None
    else:
        lcl_height = (height[:-1]*weights).sum(dim=vdim)                   # Height of LCL (AGL)
    
    del(dalr,vp,ccmr,dif,ddif,dp,ddd,eee,w_below,weights,plcl_sign,mm,malr)
    gc.collect()

    return profile,lcl_pressure,lcl_temperature,lcl_height

def thermo_quants(pressure,temperature,profile,lcl_pressure,height=None):
    """
    Calculates the following thermodynamic quantities:
    1. Convective Inhibition (CIN)
    2. Convective Avaiable Potential Energy (CAPE)
    3. Level of Free Convection (LFC)
    4. Equilibrium Level (EL)
    
    It is assumed that the first dimension in arrays is the vertical dimension
    Run `parcel_profile` first!
    
    Required inputs:
        pressure        = (array) Barometric pressure profile [Pa]
        temperature     = (array) Air temperature profile [K]
        profile         = (array) Vertical parcel temperature profile for adiabatic ascent from lowest level [K]
        lcl_pressure    = (scalar or array) Pressure at LCL [Pa] 
    
    Optional inputs:
        height          = (array) Heights profile (geopotential or AGL) corresponding to pressure levels [m]
        
    Outputs:
        cin             = (scalar or array) CIN below LFC [J/kg]
        cape            = (scalar or array) CAPE between LFC and EL [J/kg]
        lfc_pressure    = (scalar or array) Pressure at LFC [Pa]
        lfc_temperature = (scalar or array) Temperature at LFC [K]
        el_pressure     = (scalar or array) Pressure at EL [Pa]
        el_temperature  = (scalar or array) Temperature at EL [K]
        
    Optional outputs:
        lfc_height      = (scalar or array) If heights are supplied as an input, height of LFC [m]
        el_height       = (scalar or array) If heights are supplied as an input, height of EL [m]
    """
    
    global b_lcl,lfc_dif,ddif,b_buoy,s_tran,s_conv,b_cape,b_cin
    
    #vdim = pressure.dims[0]
    vdim = pressure.dims[0]   # Changed for SCAM
    shifts = {vdim: 1}
    
    dp = pressure.diff(vdim,n=1,label='lower')             # Pressure intervals between vertical levels
    dlp = np.log(pressure).diff(vdim,n=1,label='lower')    # ln(pressure) intervals between vertical levels

    # Find LCL information
    lcl_dif = -(pressure - lcl_pressure)                   # Zero at LCL, negative below, positive above
    ddif = lcl_dif.diff(vdim,n=1,label='lower')            # Intervals of differences between profiles
    b_lcl = np.heaviside(lcl_dif,0)                        # 1 above LCL, 0 below

    # Find the CAPE and CIN regions
    lfc_dif = (profile - temperature)                      # Zero at LFC, EL; pos. where parcel buoyant, neg. where not 
    ddif = lfc_dif.diff(vdim,n=1,label='lower')            # Intervals of differences between profile & environment
    b_buoy = np.heaviside(lfc_dif,1)                       # 1 in buoyant levels, 0 where not
    s_tran = np.heaviside(lfc_dif,1).diff(vdim,n=1,label='lower')  # 1 below transition to buoyant, -1 opposite transition, 0 elsewhere
    s_conv = s_tran*b_lcl[:-1]                             # Transitions only above LCL between parcel trajectory and temperature profile
    b_cape = b_lcl*b_buoy                                  # CAPE region = 1, only above LCL
#    b_cin = (1-b_cape)*(1-b_buoy.where(b_buoy[vdim]<132))  # CIN region = 1, limit to below ~600 hPa
    b_cin = (1-b_cape)*(1-b_buoy.where(press[vdim]>600.0))  # CIN region = 1, limit to below ~600 hPa

    # Find the EL
    iii = np.abs(s_conv).sum(dim=vdim)                     # Total number of transitions above LCL
    l_el = b_cape[:-1] * ((np.abs(s_conv).cumsum(dim=vdim)-s_conv)/(iii+1)).astype('int')   # 1 at level below EL, 0 elsewhere
    a_el = l_el.shift(shifts,fill_value=0.0)                                                # 1 at level just above EL, 0 elsewhere
    l_wgt = (l_el*(ddif+lfc_dif[:-1])/ddif).fillna(0)                   # Weight to pressure level below EL
    weights = l_wgt + a_el * (1-l_wgt).shift(shifts,fill_value=0)       # Weights at the layer above and below only
    el_pressure = np.exp((np.log(pressure[:-1])*weights).sum(dim=vdim)) # Pressure at EL
    el_pressure = el_pressure / np.sign(el_pressure)            # Change 0 to nan
    el_temperature = (pprof[:-1]*weights).sum(dim=vdim)                 # Temperature at EL
    el_temperature = el_temperature / np.sign(el_temperature)   # Change 0 to nan
    if height.all() == None:
        el_height = None
    else:
        el_height = (height[:-1]*weights).sum(dim=vdim)                  # Height of EL
        el_height = el_height / np.sign(el_height)              # Change 0 to nan

    # Find the LFC
    l_lfc = s_conv * ((s_conv-np.abs(s_conv).cumsum(dim=vdim)+iii+1)/(iii+1)).astype('int') # 1 at level below LFC, 0 elsewhere
    a_lfc = l_lfc.shift(shifts,fill_value=0.0)                                              # 1 at level just above LFC, 0 elsewhere
    l_wgt = l_lfc*(ddif+lfc_dif[:-1])/ddif                               # Weight to pressure level below LFC
    weights = l_wgt + a_lfc * (1-l_wgt).shift(shifts,fill_value=0)       # Weights at the layer above and below only
    lfc_pressure = np.exp((np.log(pressure[:-1])*weights).sum(dim=vdim)) # Pressure at LFC
    lfc_pressure = lfc_pressure / np.sign(np.log(lfc_pressure))    # Change 1 to inf
    lfc_temperature = (pprof[:-1]*weights).sum(dim=vdim)                 # Temperature at LFC
    lfc_temperature = lfc_temperature / np.sign(lfc_temperature)   # Change 0 to nan
    if height.all() == None:
        lfc_height = None
    else:
        lfc_height = (height[:-1]*weights).sum(dim=vdim)                 # Height of LFC
        lfc_height = lfc_height / np.sign(lfc_height)              # Change 0 to nan

    # Finally, calculate CAPE and CIN
    cin  =  R_d * (b_cin[:-1]  * lfc_dif[:-1] * dlp).sum(dim='bottom_top') * np.nan_to_num(lfc_pressure/lfc_pressure)
    cape = -R_d * (b_cape[:-1] * lfc_dif[:-1] * dlp).sum(dim='bottom_top') * np.nan_to_num(lfc_pressure/lfc_pressure)
    #if not np.isfinite(lfc_pressure):                              # No CIN if no LFC
    #    cin = cin - cin

    #del(dp,dlp,lcl_dif,ddif,b_lcl,lfc_dif,b_buoy,s_tran,s_conv,b_cape,b_cin,iii,l_el,a_el,l_wgt,weights,l_lfc,a_lfc)
    gc.collect()

    return cin,cape,lfc_pressure,lfc_temperature,el_pressure,el_temperature,lfc_height,el_height

def newvar(fullike,varname,anew_dict,adel_dict,fillval=np.nan):
    """
    Creates a new Xarray DataArray, full of nans, with given variable name and attributes, shaped like given variable
    
    Required inputs:
        fullike         = (array) Array to duplicate
        varname         = (string) Internal variable name (usually matches the DataArray name used)
        anew_dict       = (dictionary) Attributes to add
        adel_dict       = (list of strings) Names of attributes to delete
    
    Optional inputs:
        fillval         = (float, default = NaN) Value to fill array with 

    Output:
        Returns the new empty Xarray DataArray 
    """

    da = xr.full_like(fullike,fillval).load()
    da = da.rename(varname)
    da = da.assign_attrs(anew_dict)
    for attr in adel_dict:
        try:
            del da.attrs[attr]
        except:
            print(f"No attribute {attr} to delete for {varname}")
    
    return da


##############################################################################################################
##############################################################################################################
# Copyright (c) 2024 MetPy Developers.
# Distributed under the terms of the BSD 3-Clause License.
# SPDX-License-Identifier: BSD-3-Clause
"""
Contains a collection of boundary layer height estimations.

References
----------
[Col14]: Collaud Coen, M., Praz, C., Haefele, A., Ruffieux, D., Kaufmann, P., and Calpini, B. (2014)
    Determination and climatology of the planetary boundary layer height above the Swiss plateau by in situ and remote sensing measurements as well as by the COSMO-2 model
    Atmos. Chem. Phys., 14, 13205–13221.

[HL06]: Hennemuth, B., & Lammert, A. (2006):
    Determination of the atmospheric boundary layer height from radiosonde and lidar backscatter.
    Boundary-Layer Meteorology, 120(1), 181-200.

[Guo16]: Guo, J., Miao, Y., Zhang, Y., Liu, H., Li, Z., Zhang, W., ... & Zhai, P. (2016)
    The climatology of planetary boundary layer height in China derived from radiosonde and reanalysis data.
    Atmos. Chem. Phys, 16(20), 13309-13319.

[Sei00]: Seidel, D. J., Ao, C. O., & Li, K. (2010)
    Estimating climatological planetary boundary layer heights from radiosonde observations: Comparison of methods and uncertainty analysis.
    Journal of Geophysical Research: Atmospheres, 115(D16).
    
[VH96]: Vogelezang, D. H. P., & Holtslag, A. A. M. (1996)
    Evaluation and model impacts of alternative boundary-layer height formulations.
    Boundary-Layer Meteorology, 81(3-4), 245-269.
"""
#import numpy as np
from copy import deepcopy

#import metpy.calc as calc
#import metpy.constants as const
#from metpy.units import units


def smooth(val, span):
    """Function that calculates the moving average with a given span.
    The span is given in number of points on which the average is made.

    Parameters
    ----------
    val: array-like
        Array of values
    span: int
        Span of the moving average. The higher the smoother

    Returns
    -------
    smoothed_val: array-like
        Array of smoothed values

    See also
    --------
    [`bottleneck.move_mean`](https://bottleneck.readthedocs.io/en/latest/reference.html#bottleneck.move_mean), 
    [`scipy.ndimage.uniform_filter1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.uniform_filter1d.html#scipy.ndimage.uniform_filter1d), 
    [`pandas.DataFrame.rolling`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html)
    """
    n = len(val)
    smoothed_val = deepcopy(val)
    for i in range(n):
        smoothed_val[i] = np.nanmean(val[i - min(span, i) : i + min(span, n - i)])

    return smoothed_val


def bulk_richardson_number(
    height,
    potential_temperature,
    u,
    v,
    idxfoot: int = 0,
    ustar=0 * units.meter_per_second,
):
    r"""Calculate the bulk Richardson number.

    See [VH96], eq. (3):

    .. math::   Ri = (g/\theta) * \frac{(\Delta z)(\Delta \theta)}
             {\left(\Delta u)^2 + (\Delta v)^2 + b(u_*)^2}

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    u : `pint.Quantity`
        Zonal wind profile
    v : `pint.Quantity`
        Meridional wind profile
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    `pint.Quantity`
        Bulk Richardson number profile
    """
    if idxfoot == 0:
        # Force the ground level to have null wind
        Du = u
        Dv = v
    else:
        Du = u - u[idxfoot]
        Dv = v - v[idxfoot]
    
    Dtheta = potential_temperature - potential_temperature[idxfoot]
    Dz = height - height[idxfoot]

    idx0 = Du**2 + Dv**2 + ustar**2 == 0
    if idx0.sum() > 0:
        bRi = np.ones_like(Dtheta) * np.nan * units.dimensionless
        bRi[~idx0] = (
            (const.g / potential_temperature[~idx0])
            * (Dtheta[~idx0] * Dz[~idx0])
            / (Du[~idx0] ** 2 + Dv[~idx0] ** 2 + ustar**2)
        )
    else:
        bRi = (
            (const.g / potential_temperature)
            * (Dtheta * Dz)
            / (Du**2 + Dv**2 + ustar**2)
        )

    return bRi


def blh_from_richardson_bulk(
    height,
    potential_temperature,
    u,
    v,
    smoothingspan: int = 10,
    idxfoot: int = 0,
    bri_threshold=0.25 * units.dimensionless,
    ustar=0.1 * units.meter_per_second,
):
    """Calculate atmospheric boundary layer height with the method of
    bulk Richardson number.

    It is the height where the bulk Richardson number exceeds a given threshold.
    Well indicated for unstable boundary layers. See [VH96, Sei00, Col14, Guo16].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    u : `pint.Quantity`
        Zonal wind profile
    v : `pint.Quantity`
        Meridional wind profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.
    bri_threshold : `pint.Quantity`, optional
        Threshold to exceed to get boundary layer top. Defaults to 0.25
    ustar : `pint.Quantity`, optional
        Additional friction term in [VH96]. Defaluts to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    bRi = bulk_richardson_number(
        height,
        smooth(potential_temperature, smoothingspan),
        smooth(u, smoothingspan),
        smooth(v, smoothingspan),
        idxfoot=idxfoot,
        ustar=ustar,
    )

    height = height[~np.isnan(bRi)]
    bRi = bRi[~np.isnan(bRi)]

    if any(bRi > bri_threshold):
        ### iblh = np.where(bRi > bri_threshold)[0][0]  
        ### blh = height[iblh]
        ### PAD: Replace with a linear interp in z for blh
        iblh = np.searchsorted(bRi,bri_threshold)
        blh = (bri_threshold - bRi[iblh])/(bRi[iblh-1]-bRi[iblh])*(height[iblh-1]-height[iblh]) + height[iblh]
    else:
        blh = np.nan * units.meter

    return blh


def blh_from_parcel(
    height,
    potential_temperature,
    smoothingspan: int = 5,
    theta0=None,
):
    """Calculate atmospheric boundary layer height with the "parcel method"
    (or "potential temperature threshold method").

    It is the height where the potential temperature profile exceeds its
    foot value. Well indicated for unstable boundary layers. See [Sei00, HL06, Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    potential_temperature : `pint.Quantity`
        Potential temperature profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    theta0 : `pint.Quantity`, optional
        Value of theta at the foot point (skip unstruted points or add extra term). If not provided, theta[0] is taken.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    potential_temperature = smooth(potential_temperature, smoothingspan)

    if theta0 is None:
        theta0 = potential_temperature[0]

    if any(potential_temperature > theta0):
        iblh = np.where(potential_temperature > theta0)[0][0]
        blh = height[iblh]
    else:
        blh = np.nan * units.meter

    return blh


def blh_from_concentration_gradient(
    height,
    concentration_profile,
    smoothingspan: int = 5,
    idxfoot: int = 0,
):
    """Calculate atmospheric boundary layer height from a concentration
    profile (specific/relative humidity, aerosol backscatter, TKE..)

    It is the height where the gradient of the concentration profile reaches a minimum.
    Well indicated for stable boundary layers. See [Sei00, HL06, Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    concentration_profile : `pint.Quantity`
        Concentration profile (specific/relative humidity, aerosol backscatter, TKE..)
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    dcdz = calc.first_derivative(smooth(concentration_profile, smoothingspan), x=height)
    dcdz = dcdz[idxfoot:]
    height = height[idxfoot:]
    iblh = np.argmin(dcdz)

    return height[iblh]


def blh_from_temperature_inversion(
    height,
    temperature,
    smoothingspan: int = 5,
    idxfoot: int = 0,
):
    """Calculate atmospheric boundary layer height from the inversion of
    absolute temperature gradient

    It is the height where the temperature gradient (absolute or potential) changes of sign.
    Well indicated for stable boundary layers. See [Col14].

    Parameters
    ----------
    height : `pint.Quantity`
        Altitude (metres above ground) of the points in the profile
    humidity : `pint.Quantity`
        Temperature (absolute or potential) profile
    smoothingspan : int, optional
        The amount of smoothing (number of points in moving average)
    idxfoot : int, optional
        The index of the foot point (first trusted measure), defaults to 0.

    Returns
    -------
    blh : `pint.Quantity`
        Boundary layer height estimation
    """
    dTdz = calc.first_derivative(smooth(temperature, smoothingspan), x=height)

    dTdz = dTdz[idxfoot:]
    height = height[idxfoot:]

    if any(dTdz * dTdz[0] < 0):
        iblh = np.where(dTdz * dTdz[0] < 0)[0][0]
        blh = height[iblh]
    else:
        blh = np.nan * units.meter

    return blh

## Computational loop

In [ ]:
############################################################
timer0 = time.perf_counter()
for expo in exp_list[ex0:ex1]:
    case = expo.split('/')[-1][:-3]
    yyyy, mm, dd = case[4:8], case[8:10], case[10:12]
    print(f"Case {case[4:]}: ",end="")
    #sys.exit(0)
    
    #### Make array of date-time groups for the Time dimension
    dtg = np.array([],dtype=np.datetime64)
    filelist = []
    for h in hours:
        if h[0] == "0":
            if int(dd) == dom[int(mm)-1]:
                dd = "00"
                mm = f"{(int(mm) % 12) + 1:02d}"
            filelist.append(f"{expo}/diag_d01_{yyyy}-{mm}-{(int(dd)+1):02d}_{h}:00:00{suffix}")                
            dtg = np.append(dtg,np.datetime64(f"{yyyy}-{mm}-{(int(dd)+1):02d}T{h}:00"))
        else:
            filelist.append(f"{expo}/diag_d01_{yyyy}-{mm}-{dd}_{h}:00:00{suffix}")
            dtg = np.append(dtg,np.datetime64(f"{yyyy}-{mm}-{dd}T{h}:00"))
    les_data = xr.open_mfdataset(filelist,combine="nested",concat_dim="Time",decode_times=False)
    
    #### Initialize new Xarray DataArrays
    # For the 2-D domain
    AVS_REGIME = newvar(les_data.AVS_PSFC,"AVS_REGIME",
                        {"units":"-","description":"convective regime","key":"0 = neutral, 1 = stable, -1 = unstable"},
                        ['FieldType','MemoryOrder','stagger'])
    AVS_CBP    = newvar(les_data.AVS_PSFC,"AVS_CBP",
                        {"units":"Pa","description":"pressure at cloud base"},
                        ['FieldType','MemoryOrder','stagger'])
    AVS_CBZ    = newvar(les_data.AVS_PSFC,"AVS_CBZ",
                        {"units":"m","description":"height above ground of cloud base"},
                        ['FieldType','MemoryOrder','stagger'])
    AVS_PBLP   = newvar(les_data.AVS_PSFC,"AVS_PBLP",
                        {"units":"Pa","description":"pressure at PBL top based on potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_PBLZ   = newvar(les_data.AVS_PSFC,"AVS_PBLZ",
                        {"units":"m","description":"height above ground of PBL top based on potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_PBLPV  = newvar(les_data.AVS_PSFC,"AVS_PBLPV",
                        {"units":"Pa","description":"pressure at PBL top based on virtual potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_PBLZV  = newvar(les_data.AVS_PSFC,"AVS_PBLZV",
                        {"units":"m","description":"height above ground of PBL top based on virtual potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LCLZ   = newvar(les_data.AVS_PSFC,"AVS_LCLZ",
                        {"units":"m","description":"height above ground of the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LCLP   = newvar(les_data.AVS_PSFC,"AVS_LCLP",
                        {"units":"Pa","description":"pressure at the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LCLT   = newvar(les_data.AVS_PSFC,"AVS_LCLT",
                        {"units":"K","description":"temperature at the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LFCZ   = newvar(les_data.AVS_PSFC,"AVS_LFCZ",
                        {"units":"m","description":"height above ground of the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LFCP   = newvar(les_data.AVS_PSFC,"AVS_LFCP",
                        {"units":"Pa","description":"pressure at the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_LFCT   = newvar(les_data.AVS_PSFC,"AVS_LFCT",
                        {"units":"K","description":"temperature at the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_ELZ    = newvar(les_data.AVS_PSFC,"AVS_ELZ",
                        {"units":"m","description":"height above ground of the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_ELP    = newvar(les_data.AVS_PSFC,"AVS_ELP",
                        {"units":"Pa","description":"pressure at the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_ELT    = newvar(les_data.AVS_PSFC,"AVS_ELT",
                        {"units":"K","description":"temperature at the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_CIN    = newvar(les_data.AVS_PSFC,"AVS_CIN",
                        {"units":"J/kg","description":"convective inhibition"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVS_CAPE   = newvar(les_data.AVS_PSFC,"AVS_CAPE",
                        {"units":"J/kg","description":"convective available potential energy"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    
    # For the domain average
    AVT_REGIME = newvar(les_data.AVT_PSFC,"AVT_REGIME",
                        {"units":"-","description":"convective regime","key":"0 = neutral, 1 = stable, -1 = unstable"},
                        ['FieldType','MemoryOrder','stagger'])
    AVT_CBP    = newvar(les_data.AVT_PSFC,"AVT_CBP",
                        {"units":"Pa","description":"pressure at cloud base"},
                        ['FieldType','MemoryOrder','stagger'])
    AVT_CBZ    = newvar(les_data.AVT_PSFC,"AVT_CBZ",
                        {"units":"m","description":"height above ground of cloud base"},
                        ['FieldType','MemoryOrder','stagger'])
    AVT_PBLP   = newvar(les_data.AVT_PSFC,"AVT_PBLP",
                        {"units":"Pa","description":"pressure at PBL top based on potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_PBLZ   = newvar(les_data.AVT_PSFC,"AVT_PBLZ",
                        {"units":"m","description":"height above ground of PBL top based on potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_PBLPV  = newvar(les_data.AVT_PSFC,"AVT_PBLPV",
                        {"units":"Pa","description":"pressure at PBL top based on virtual potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_PBLZV  = newvar(les_data.AVT_PSFC,"AVT_PBLZV",
                        {"units":"m","description":"height above ground of PBL top based on virtual potential temperature profiles","details":"Liu and Liang (2010) method used to estimate the PBL height"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LCLZ   = newvar(les_data.AVT_PSFC,"AVT_LCLZ",
                        {"units":"m","description":"height above ground of the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LCLP   = newvar(les_data.AVT_PSFC,"AVT_LCLP",
                        {"units":"Pa","description":"pressure at the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LCLT   = newvar(les_data.AVT_PSFC,"AVT_LCLT",
                        {"units":"K","description":"temperature at the lifted condensation level (LCL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LFCZ   = newvar(les_data.AVT_PSFC,"AVT_LFCZ",
                        {"units":"m","description":"height above ground of the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LFCP   = newvar(les_data.AVT_PSFC,"AVT_LFCP",
                        {"units":"Pa","description":"pressure at the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_LFCT   = newvar(les_data.AVT_PSFC,"AVT_LFCT",
                        {"units":"K","description":"temperature at the level of free convection (LFC)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_ELZ    = newvar(les_data.AVT_PSFC,"AVT_ELZ",
                        {"units":"m","description":"height above ground of the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_ELP    = newvar(les_data.AVT_PSFC,"AVT_ELP",
                        {"units":"Pa","description":"pressure at the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_ELT    = newvar(les_data.AVT_PSFC,"AVT_ELT",
                        {"units":"K","description":"temperature at the equilibrium level (EL)"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_CIN    = newvar(les_data.AVT_PSFC,"AVT_CIN",
                        {"units":"J/kg","description":"convective inhibition"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    AVT_CAPE   = newvar(les_data.AVT_PSFC,"AVT_CAPE",
                        {"units":"J/kg","description":"convective available potential energy"},
                        ['FieldType','MemoryOrder','stagger'],fillval=0.0)
    #sys.exit(0)

    ####### Loop through hours
    for t,h in enumerate(hours):  ######################################################################
        print(h,end="Z")
        
        ##### First for the 2-D fields
        # Extract timesteps and set to local variables for convenience
        p_s   = les_data.AVS_PSFC[t].values                               # Surface pressure
        press  = les_data.AVV_P[t,:]                                      # Pressure profile
        press.load()          # These `load`s are to speed up processing
        height = les_data.AVV_Z[t,:]                                      # Height AGL
        height.load()
        rho = les_data.AVV_RHO[t,:]                                       # Air density
        rho.load()
        theta = les_data.AVV_TH[t,:]                                      # Potential temperature profile  
        theta.load()
        thetav = les_data.AVV_THV[t,:]                                    # Virtual potential temperature profile    
        thetav.load()

        tk = les_data.AVV_TK[t,:]                                         # Air Temperature
        tk.load()
        qv = les_data.AVV_QV[t,:]                                         # Water vapor mixing ratio
        qv.load()
        #td = calc.dewpoint_from_specific_humidity(press.values * units('Pa'),
        #            tk.values * units('K'),
        #            calc.specific_humidity_from_mixing_ratio(qv.values)).to(units('K')).magnitude
       
        qc = les_data.AVV_QC[t,:]                                         # Cloud mixing ratio
        qc.load()
        
        ######## Parcel quantities estimated
        pprof,p_lcl,t_lcl,z_lcl = parcel_profile(press,tk,qv[0],height)  # Find LCL, parcel profile
        print("⋅",end="")
        #sys.exit(0)
        cin,cape,p_lfc,t_lfc,p_el,t_el,z_lfc,z_el = thermo_quants(press,tk,pprof,p_lcl,height)  # Find LFC
        print(":",end="")
        ######## Cloud base estimated
        p_wcb,z_wcb = cloud_base(qc,press,height,min_qc=1.0e-5)
        print("-",end="")

        ######## PBL heights estimated
        # Here we don't use the bulk Richardson number approach, as the 3-D wind fields are massive
        p_pbl,z_pbl,regime = pbl_from_pottemp(press,theta,height,delta_s=delta_s,delta_u=delta_u,delta_p=delta_p,overshoot=overshoot)
        p_pblv,z_pblv,dummy = pbl_from_pottemp(press,thetav,height,delta_s=delta_s,delta_u=delta_u,delta_p=delta_p,overshoot=overshoot)       
        print("=",end="")
        
        ##### Fill DataArray
        AVS_REGIME[t] = regime
        AVS_CBP[t] = p_wcb
        AVS_CBZ[t] = z_wcb
        AVS_PBLP[t] = p_pbl
        AVS_PBLZ[t] = z_pbl
        AVS_PBLPV[t] = p_pblv
        AVS_PBLZV[t] = z_pblv
        AVS_LCLP[t] = p_lcl
        AVS_LCLZ[t] = z_lcl
        AVS_LCLT[t] = t_lcl
        AVS_LFCP[t] = p_lfc
        AVS_LFCZ[t] = z_lfc
        AVS_LFCT[t] = t_lfc
        AVS_ELP[t] = p_el
        AVS_ELZ[t] = z_el
        AVS_ELT[t] = t_el
        AVS_CIN[t] = cin
        AVS_CAPE[t] = cape

        ##### Then the domain average
        # Extract timesteps and set to local variables for convenience
        p_s   = les_data.AVT_PSFC[t].values                               # Surface pressure
        press  = les_data.AVP_P[t,:]                                      # Pressure profile
        press.load()          # These `load`s are to speed up processing
        height = les_data.AVP_Z[t,:]                                      # Height AGL
        height.load()
        rho = les_data.AVP_RHO[t,:]                                       # Air density
        rho.load()
        theta = les_data.AVP_TH[t,:]                                      # Potential temperature profile  
        theta.load()
        thetav = les_data.AVP_THV[t,:]                                    # Virtual potential temperature profile    
        thetav.load()
        uwind = les_data.AVP_U[t,:]                                       # Zonal wind
        uwind.load()
        vwind = les_data.AVP_V[t,:]                                       # Meridional wind
        vwind.load()
        
        tk = les_data.AVP_TK[t,:]                                         # Air Temperature
        tk.load()
        qv = les_data.AVP_QV[t,:]                                         # Water vapor mixing ratio
        qv.load()
        #td = calc.dewpoint_from_specific_humidity(press.values * units('Pa'),
        #            tk.values * units('K'),
        #            calc.specific_humidity_from_mixing_ratio(qv.values)).to(units('K')).magnitude
       
        qc = les_data.AVP_QC[t,:]                                         # Cloud mixing ratio
        qc.load()
        ######## Parcel quantities estimated
        pprof,p_lcl,t_lcl,z_lcl = parcel_profile(press,tk,qv[0],height)  # Find LCL, parcel profile
        #print("_",end="")
        #sys.exit(0)
        cin,cape,p_lfc,t_lfc,p_el,t_el,z_lfc,z_el = thermo_quants(press,tk,pprof,p_lcl,height)  # Find LFC
        #print("_",end="")
        ######## Cloud base estimated
        p_wcb,z_wcb = cloud_base(qc,press,height,min_qc=1.0e-5)
        #print("_",end="")

        ######## PBL heights estimated
        ###p_pbl,z_pbl,regime = pbl_from_pottemp(press,theta,height,delta_s=delta_s,delta_u=delta_u,delta_p=delta_p,overshoot=overshoot)
        ###p_pblv,z_pblv,dummy = pbl_from_pottemp(press,thetav,height,delta_s=delta_s,delta_u=delta_u,delta_p=delta_p,overshoot=overshoot)
        # To replace above - use bulk Richardson approach when we have the winds
        z_pbl = blh_from_richardson_bulk(height.data*units('meter'),theta.data*units('kelvin'),uwind.data*units('m/s'),vwind.data*units('m/s'),smoothingspan=1)
        z_pbl = z_pbl.m
        ii = np.searchsorted(height.data,z_pbl)
        p_pbl = np.exp((z_pbl - height[ii])/(height[ii-1]-height[ii])*np.log(press[ii-1]/press[ii]) + np.log(press[ii]))
        z_pblv = blh_from_richardson_bulk(height.data*units('meter'),thetav.data*units('kelvin'),uwind.data*units('m/s'),vwind.data*units('m/s'),smoothingspan=1)
        z_pblv = z_pblv.m
        ii = np.searchsorted(height.data,z_pblv)
        p_pblv = np.exp((z_pblv - height[ii])/(height[ii-1]-height[ii])*np.log(press[ii-1]/press[ii]) + np.log(press[ii]))
        #print("_",end="")
        
        ##### Fill DataArray
        AVT_REGIME[t] = np.nan
        AVT_CBP[t] = p_wcb
        AVT_CBZ[t] = z_wcb
        AVT_PBLP[t] = p_pbl
        AVT_PBLZ[t] = z_pbl
        AVT_PBLPV[t] = p_pblv
        AVT_PBLZV[t] = z_pblv
        AVT_LCLP[t] = p_lcl
        AVT_LCLZ[t] = z_lcl
        AVT_LCLT[t] = t_lcl
        AVT_LFCP[t] = p_lfc
        AVT_LFCZ[t] = z_lfc
        AVT_LFCT[t] = t_lfc
        AVT_ELP[t] = p_el
        AVT_ELZ[t] = z_el
        AVT_ELT[t] = t_el
        AVT_CIN[t] = cin
        AVT_CAPE[t] = cape

    # Combine data arrays into a dataset
    print("\033[1m output\033[0m",end="")
    o_ds = xr.merge([AVS_REGIME,AVS_CBP,AVS_CBZ,
                     AVS_PBLP,AVS_PBLZ,AVS_PBLPV,AVS_PBLZV,
                     AVS_LCLP,AVS_LCLZ,AVS_LCLT,
                     AVS_LFCP,AVS_LFCZ,AVS_LFCT,
                     AVS_ELP,AVS_ELZ,AVS_ELT,
                     AVS_CIN,AVS_CAPE,
                     AVT_REGIME,AVT_CBP,AVT_CBZ,
                     AVT_PBLP,AVT_PBLZ,AVT_PBLPV,AVT_PBLZV,
                     AVT_LCLP,AVT_LCLZ,AVT_LCLT,
                     AVT_LFCP,AVT_LFCZ,AVT_LFCT,
                     AVT_ELP,AVT_ELZ,AVT_ELT,
                     AVT_CIN,AVT_CAPE])
    o_ds = o_ds.assign_coords({"Time": dtg})
    #sys.exit(0)

    encoding = {var: deflate for var in o_ds.data_vars}
    o_ds.to_netcdf(path=f"{pdir}{prefix}_{case[4:]}.nc4",engine="netcdf4",format="netCDF4",encoding=encoding)
    timer1 = time.perf_counter()
    print(f" {(timer1-timer0)/60:.0f} min")
    timer0=timer1
    #sys.exit(0)
print("***DONE***")